In [1]:
import json
import os
import pandas as pd
import urllib.request
import time
import swifter
import datetime

## Load Aggregated Report from File and download the jsons from links

In [2]:
run1="Aggregated_Report1722845"
run3="Aggregated_Report_1726283"
run4="Aggregated_Report_1726283_2"
run5="Aggregated_Report_1726283_3"
run6="Aggregated_Report_1734122"

filename = run6

agg_report = pd.read_csv("appen_results/" + filename + ".csv")
print(agg_report.shape)
print(agg_report.columns)

(417, 12)
Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'annotations', 'annotations:confidence',
       'annotations_gold', 'clean_description', 'dream_id', 'name', 'p_id'],
      dtype='object')


In [3]:
print('#AMT NAN annotations:', agg_report.annotations.isna().sum()) # Remove those rows with a NAN annotation..
agg_report = agg_report[agg_report.annotations.notna()].copy()
agg_report.shape    


#AMT NAN annotations: 0


(417, 12)

In [4]:
%%time
#The annotations are for some reasons only embedded as link --> extra download needed.
def download_json(row):
    #print('Load Unit ID:', row['_unit_id'], ' row_index: ', row.name)
    data = urllib.request.urlopen(row['annotations']).read()
    output = json.loads(data)
    return output

#And Persist Results, since we don't wanna do this every time...
if not os.path.exists('intermediate_store/' + filename + '.pickle'):
    agg_report["anno_json"] = agg_report.apply(lambda row: download_json(row), axis=1)
    agg_report.to_pickle('intermediate_store/' + filename + '.pickle')
    print("Created new Pickle File")
else:
    agg_report = pd.read_pickle('intermediate_store/' + filename + '.pickle')
    print("Loaded Pickle File")


Created new Pickle File
CPU times: user 18.5 s, sys: 644 ms, total: 19.1 s
Wall time: 8min 48s


# Remove gold data and report Confidence

In [5]:
gold_data = agg_report[agg_report['_golden'] == True].copy()
new_data = agg_report[agg_report['_golden'] == False].copy()

print('Overall confidence mean: ', agg_report['annotations:confidence'].mean())
print('Gold Data confidence mean: ', gold_data['annotations:confidence'].mean())
print('New Data confidence mean: ', new_data['annotations:confidence'].mean())


Overall confidence mean:  0.7390856115107914
Gold Data confidence mean:  0.8162367521367521
New Data confidence mean:  0.7089966666666666


# Remove old and gold annotations  

### TODO UPDATE DATE FOR EACH RUN 

In [6]:
agg_report._last_judgment_at = pd.to_datetime(agg_report._last_judgment_at)
comparsion_date = datetime.datetime(2021,2,20,0,0,0) # year, month, day etc..
agg_report = agg_report[agg_report['_last_judgment_at'] >=  comparsion_date ].copy()
agg_report = agg_report[agg_report['_golden'] ==  False ].copy()
agg_report.shape


(300, 13)

In [7]:
agg_report.head(10).name

0             ClearJuice Cartridges 510 Thread FE Only
1    New!!sample listing Blotters LSD 220ug 2x (Fre...
2    (Quantity x10,000) 2.5mg Xanax Bar RxPress G37...
3                                   100g Heroin Afghan
4      Genuine Pfizer Cytotec - 50 tablets - Free Ship
5                    3.5g  Premium Royal Moroccan  30 
6                                5 grams / Malana Hash
7       "1oz. ""Purple Sizzurp"" codeine/promethazine"
8                         1g Mr. Suprise Blok [AAA+++]
9                       1x 65mg THC Organic Nerds Rope
Name: name, dtype: object

# Load Labelstudio Example format

In [8]:
#with open('appen_results/label_studio_example.json') as json_file:
 #   label_studio = json.load(json_file)
#label_studio[3]['completions'][0]['result']

In [9]:
# Toplevel: dict_keys(['completions', 'data', 'id'])
# Completions 
    #--> List --> only one for the beginning [0]
        # dict_keys(['created_at', 'id', 'lead_time', 'result'])
            #created_at --> Unix time
            #id --> unit_id from appen
            #lead_time --> I don't know set to default
            # Result --> List of all label dicts
                #Dict Items:
                    #From_name: No annotator ID --> default 'Appen'
                    # id --> span_id from annotation
                    #to_name: 'text'
                    # type : 'labels'
                    # value : dict
                        #start: char id
                        #end: char id
                        #labels:['Drug']
                        #text: text of token/full span
#data:dict
    #dream_id
    #p_id
    #description
    #name
#id:unique id for labelstudio.    

                    

In [10]:
def get_span_annotation(span_dict, classname):
     #initialize tokenspan min range
    startIdx=span_dict['tokens'][0]['startIdx']
    endIdx=span_dict['tokens'][0]['endIdx']
    text=''

    #Get start and end of token span
    for token_dict in span_dict['tokens']:
        if token_dict['startIdx'] < startIdx:
            startIdx = token_dict['startIdx']
        if token_dict['endIdx'] > endIdx:
            endIdx = token_dict['endIdx']
        text = text + ' ' +   token_dict['text']
        
    return {'from_name': 'label',
                       'id': span_dict['id'],
                       'to_name': 'text',
                       'type': 'labels',
                       'value':{'end':endIdx,
                                    'labels': [classname],
                                    'text':text.strip(),
                                    'start':startIdx}
    }

def get_result_json(json, ind):
    result=[]
    #print(ind)
    for span_dict in json['spans']:
        if span_dict['annotated_by'] == 'human' and len(span_dict['classnames']) > 0:
            if len(span_dict['classnames']) > 1:
                print('WTF is that!', span_dict['classnames'])
            
            elif span_dict['classnames'][0] == 'Drug' :
                result.append(get_span_annotation(span_dict, 'Drug'))
            elif span_dict['classnames'][0] == 'None':
                result.append(get_span_annotation(span_dict, 'None'))
            else:
                print('I Found a NONE!!!!')
    return result

            
def create_labelstudio_data(agg_row):
    data={'dream_id':agg_row.dream_id, 'name':agg_row['name'], 'p_id':agg_row.p_id, 
          'clean_description':agg_row.clean_description, 'golden':agg_row['_golden'] }

    completions={'created_at': int(time.time()),
                'id':agg_row._unit_id,
                'lead_time':10.00,        #I have no clue what lead time is.
                'result':get_result_json(agg_row.anno_json, agg_row.name)
                }
    
    return {'completions':[completions], 'data':data, 'id':agg_row._unit_id}
    

agg_report['labelstudio_json'] = agg_report.apply(lambda row: create_labelstudio_data(row), axis=1)

In [11]:
agg_report.shape

(300, 14)

In [12]:
#Write TO File
with open('appen_results/label_studio_' + filename + '.json', 'w') as json_file:
    json.dump(agg_report['labelstudio_json'].to_list(), json_file, indent='\t')

In [13]:
#Investigate Confidence values.
'''for i, row in agg_report.iterrows():
    print('Overall Confidence: ', row['annotations:confidence'], ' of row ', i, ' unitID: ', row['_unit_id'])
    for span in row['anno_json']['spans']:
        if 'confidence' in span:
            print('   span Conf: ', span['confidence'])
'''

"for i, row in agg_report.iterrows():\n    print('Overall Confidence: ', row['annotations:confidence'], ' of row ', i, ' unitID: ', row['_unit_id'])\n    for span in row['anno_json']['spans']:\n        if 'confidence' in span:\n            print('   span Conf: ', span['confidence'])\n"

In [14]:
#Run1
#Overall Confidence:  0.7625  of row  140  unitID:  2945786522
#   span Conf:  0.8667
#   span Conf:  0.6583

In [15]:
agg_report['annotations:confidence'].mean()

0.7089966666666666